In [ ]:
import wandb
import pandas as pd
import matplotlib.pyplot as plt
import os
import json

In [ ]:
api = wandb.Api(timeout=20)

In [ ]:
run = api.run("pjajal/inference-diffusion-noise-optim/fv5iu18q")

In [ ]:
scan_history = run.scan_history()

In [ ]:
def flatten_log(log):
    out = {}
    for k, v in log.items():
        if isinstance(v, dict):
            for k2, v2 in v.items():
                out[f"{k}.{k2}"] = v2
        elif isinstance(v, list):
            if len(v) == 1:
                out[k] = v[0]
        else:
            out[k] = v
    return out

In [ ]:
logs = []
for log in scan_history:
    logs.append(flatten_log(log))

In [ ]:
all_data = pd.DataFrame(logs)

In [ ]:
all_data

In [ ]:
unq_prompts = {i: prompt for i, prompt in enumerate(all_data['prompt'].unique())}
reverse_unq_prompts = {v: k for k, v in unq_prompts.items()}
with open("./eval_results/fv5iu18q/prompts.json", "w") as f:
    json.dump(unq_prompts, f)

In [ ]:
# For each prompt, extract the row corresponding to the 0-th step and the step with the maximum pop_best_eval
best_worst = []
for prompt, df_group in all_data.groupby("prompt"):
    step_0_row = df_group[df_group["step"] == 0]
    max_pop_best_eval_row = df_group.loc[df_group["pop_best_eval"].idxmax()]
    idx_prompt = reverse_unq_prompts[prompt]
    
    save_loc = f"eval_results/fv5iu18q/{idx_prompt}/"
    os.makedirs(save_loc, exist_ok=True)
    for i, row in df_group.iterrows():
        step = row['step']
        img_save_loc = os.path.join(save_loc, f"{step}.png")
        if os.path.exists(img_save_loc):
            continue
        try:
            status = run.file(row['best_img.path']).download()
            os.renames(status.name, os.path.join(save_loc, f"{step}.png"))
        except:
            print(f"Failed to download {row['best_img.path']}")

    baseline_save_loc = os.path.join(save_loc, "baseline.png")
    max_save_loc = os.path.join(save_loc, "max.png")
    
    if not os.path.exists(baseline_save_loc):
        baseline_status = run.file(step_0_row['best_img.path'].item()).download()
        os.renames(baseline_status.name, os.path.join(save_loc, "baseline.png"))

    if not os.path.exists(max_save_loc):
        max_status = run.file(max_pop_best_eval_row['best_img.path']).download()
        os.renames(max_status.name, os.path.join(save_loc, "max.png"))
    best_worst.append({"prompt": prompt, "baseline": step_0_row['pop_best_eval'].item(), "best": max_pop_best_eval_row['pop_best_eval'].item()})
    print(
        f"Prompt: {prompt}, Step 0: {step_0_row['pop_best_eval'].item()}, Max Pop Best Eval: {max_pop_best_eval_row['pop_best_eval'].item()}"
    )

best_worst_results = pd.DataFrame(best_worst)
best_worst_results.to_csv("eval_results/fv5iu18q/best_worst_results.csv", index=False)

In [ ]:
from PIL import Image
import hpsv2
import torch
import ImageReward
import os
import json
from diffusers.utils import pt_to_pil, numpy_to_pil
import numpy as np
from glob import glob
from transformers import (
    CLIPProcessor,
    CLIPImageProcessor,
    CLIPModel,
    AutoModel,
    AutoProcessor,
    AutoTokenizer,
)
import pandas as pd
import torch.nn as nn

In [ ]:
img_reward = ImageReward.load("ImageReward-v1.0")
img_reward = img_reward.eval()

In [ ]:
eval_results_loc = "eval_results/fv5iu18q/"
prompts_file = os.path.join(eval_results_loc, "prompts.json")
best_worst_file = os.path.join(eval_results_loc, "best_worst_results.csv")

with open(prompts_file, "r") as f:
    prompt_dict = json.load(f)

In [ ]:
measurements = []
for idx, prompt in prompt_dict.items():
    img_save_loc = os.path.join(eval_results_loc, idx)

    path_score_list = []
    for img_path in glob(os.path.join(img_save_loc, "[0-9]*.png")):
        img = Image.open(img_path)

        img_reward_score = img_reward.score(prompt, img)
        path_score_list.append((img_path, img_reward_score))

    highest_score = max(path_score_list, key=lambda x: x[1])
    lowest_score = min(path_score_list, key=lambda x: x[1])
    
    
    baseline_score_path = os.path.join(img_save_loc, "baseline.png")
    baseline_img = Image.open(baseline_score_path)
    baseline_img_score = img_reward.score(prompt, baseline_img)

    measurements.append(
        {
            "prompt": prompt,
            "highest_score": highest_score[1],
            "highest_score_path": highest_score[0],
            "lowest_score": lowest_score[1],
            "lowest_score_path": lowest_score[0],
            "baseline_score": baseline_img_score,
            "baseline_score_path": baseline_score_path,
        }
    )

In [ ]:
pd.DataFrame(measurements).to_csv("eval_results/fv5iu18q/measurements.csv", index=False)

In [51]:
measurements = pd.read_csv("eval_results/fv5iu18q/measurements.csv")

In [52]:
processor = CLIPProcessor.from_pretrained("openai/clip-vit-large-patch14")
clip_model = CLIPModel.from_pretrained(
    "openai/clip-vit-large-patch14"
).eval().to(device="mps")

@torch.no_grad()
def clip_score(processor, clip_model, prompt, img):
    inputs = processor(text=[prompt], images=[img], return_tensors="pt", padding=True).to(device="mps")
    outputs = clip_model(**inputs)
    score = outputs[0][0]
    return score.tolist()

In [53]:
class AestheticMLP(nn.Module):
    def __init__(self, input_size: int):
        super().__init__()
        self.input_size = input_size
        self.layers = nn.Sequential(
            nn.Linear(self.input_size, 1024),
            nn.Dropout(0.2),
            nn.Linear(1024, 128),
            nn.Dropout(0.2),
            nn.Linear(128, 64),
            nn.Dropout(0.1),
            nn.Linear(64, 16),
            nn.Linear(16, 1),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        ### Normalize CLIP embedding
        l2 = torch.linalg.vector_norm(x, dim=-1, keepdim=True)
        ### NEXT LINE MESSES WITH DNO GRADIENT! There are conflicting implementations (some have this, some dont)
        ### Opting to leave it out
        # l2[l2 == 0] = 1
        x = x / l2
        ### Apply MLP, return score
        x = self.layers(x)
        return x


### Load CLIP model and processor
aesthetic_processor = CLIPProcessor(
    CLIPImageProcessor.from_pretrained("openai/clip-vit-large-patch14"),
    AutoTokenizer.from_pretrained("openai/clip-vit-large-patch14"),
)

aesthetic_clip_model = (
    CLIPModel.from_pretrained("openai/clip-vit-large-patch14").eval().to(device="mps")
)

### Load linear classifier
aesthetic_mlp = AestheticMLP(768)
aesthetic_mlp.load_state_dict(
    torch.load(os.path.join("./", "aesthetic_mlp.pth"), map_location="cpu"),
    strict=False,
    assign=True,
)
aesthetic_mlp.eval().to(device="mps")

def aesthetic_score(img):
    inputs = aesthetic_processor(
        images=img,
        return_tensors="pt",
        padding=True,
        do_rescale=False,
    ).to(device="mps")

    clip_image_embeddings = aesthetic_clip_model.get_image_features(**inputs)
    score = aesthetic_mlp(clip_image_embeddings)

    return score


In [54]:
all_measurements = []
for i, row in measurements.iterrows():
    prompt = row["prompt"]
    highest_score_img_reward = row["highest_score"]
    lowest_score_img_reward = row["lowest_score"]
    baseline_score_img_reward = row["baseline_score"]
    highest_score_path = row["highest_score_path"]
    lowest_score_path = row["lowest_score_path"]
    baseline_score_path = row["baseline_score_path"]

    highest_score_img = Image.open(highest_score_path)
    lowest_score_img = Image.open(lowest_score_path)
    baseline_score_img = Image.open(baseline_score_path)

    # HPSV2
    with torch.no_grad():
        highest_score_hpsv = hpsv2.score(highest_score_img, prompt)
        lowest_score_hpsv = hpsv2.score(lowest_score_img, prompt)
        baseline_score_hpsv = hpsv2.score(baseline_score_img, prompt)

    # CLIP
    highest_score_clip = None
    lowest_score_clip = None
    baseline_score_clip = None
    try:
        with torch.no_grad():
            highest_score_clip = clip_score(
                processor, clip_model, prompt, highest_score_img
            )
            lowest_score_clip = clip_score(
                processor, clip_model, prompt, lowest_score_img
            )
            baseline_score_clip = clip_score(
                processor, clip_model, prompt, baseline_score_img
            )
    except:
        print(f"Failed to score {prompt} with CLIP")

    # Aesthetic
    with torch.no_grad():
        highest_score_aesthetic = aesthetic_score(highest_score_img)
        lowest_score_aesthetic = aesthetic_score(lowest_score_img)
        baseline_score_aesthetic = aesthetic_score(baseline_score_img)

    all_measurements.append(
        {
            "prompt": prompt,
            "highest_score_img_reward": highest_score_img_reward,
            "lowest_score_img_reward": lowest_score_img_reward,
            "baseline_score_img_reward": baseline_score_img_reward,
            "highest_score_hpsv": highest_score_hpsv,
            "lowest_score_hpsv": lowest_score_hpsv,
            "baseline_score_hpsv": baseline_score_hpsv,
            "highest_score_clip": highest_score_clip,
            "lowest_score_clip": lowest_score_clip,
            "baseline_score_clip": baseline_score_clip,
            "highest_score_aesthetic": highest_score_aesthetic,
            "lowest_score_aesthetic": lowest_score_aesthetic,
            "baseline_score_aesthetic": baseline_score_aesthetic,
        }
    )

Failed to score An instqrumemnt used for cutting cloth, paper, axdz othr thdin mteroial, consamistng of two blades lad one on tvopb of the other and fhastned in tle mixdqdjle so as to bllow them txo be pened and closed by thumb and fitngesr inserted tgrough rings on kthe end oc thei vatndlzes. with CLIP
Failed to score A ldarge keybord msical instroument lwith a woden case enmclosig a qsouvnkboajrd and mfgtal strivgf, which are strucrk b hammrs when the nels are depresdsmed.f lhe strsingsj' vibration ie stopped by damperds when the keys re released and can bce regulavewdd for lengh and vnolume y two or three pedalvs. with CLIP


KeyboardInterrupt: 

In [ ]:
def handle_v(v):
    if isinstance(v, list):
        return handle_v(v[0])
    if isinstance(v, np.float32):
        return v.item()
    return v
    

pd.DataFrame(map(lambda x: {k: handle_v(v) for k, v in x.items()}, all_measurements)).to_csv("eval_results/fv5iu18q/all_measurements.csv", index=False)

In [50]:
all_meas = pd.read_csv("eval_results/fv5iu18q/all_measurements.csv")

In [ ]:
all_meas['high_vs_baseline_img_reward'] = all_meas['highest_score_img_reward'] - all_meas['baseline_score_img_reward']
all_meas['high_vs_baseline_hpsv'] = all_meas['highest_score_hpsv'] - all_meas['baseline_score_hpsv']
all_meas['high_vs_baseline_clip'] = all_meas['highest_score_clip'] - all_meas['baseline_score_clip']

In [ ]:
all_meas[['high_vs_baseline_img_reward', 'high_vs_baseline_hpsv', 'high_vs_baseline_clip']].mean()